# Exercise: Introduction to accelerated computing

## AXPY with KernelAbstractions

KernelAbstractions is a light-weight domain-specific language that enables you to write portable kernels.

It allows us to target many different backend packages:

- CUDA.jl
- Metal.jl
- OneAPI.jl
- AMDGPU.jl

And it provides a fallback implementation for CPU

In [21]:
import Pkg
Pkg.activate("@colab")
Pkg.add([
    "CUDA",
    "KernelAbstractions",
    "LinearAlgebra",
    "Random",
    "BenchmarkTools",
    "Adapt",
    "OffsetArrays",
    "CairoMakie",
    "OrdinaryDiffEqTsit5",
])

  Activating project at `/content/@colab`
   Resolving package versions...
  No Changes to `/content/@colab/Project.toml`
  No Changes to `/content/@colab/Manifest.toml`


In [22]:
import CUDA

The AXPY kernel is defined as

$Y = Y + a*X$

Vary the `M` below.

In [23]:
M = 1024

1024

In [24]:
function axpy_serial!(Y, a, X)
	for i in eachindex(Y, X)
		# TODO: Implement
	end
	return nothing
end

axpy_serial! (generic function with 1 method)

In [25]:
using KernelAbstractions

In [26]:
@kernel function axpy_kernel!(Y, a, X)
	# implement me!
end

In [27]:
# Creating a wrapper kernel for launching with error checks
function axpy_ka!(Y, a, X)
    if length(Y) != length(X)
		error("Arrays disagree in length")
        return nothing
    end
    backend = KernelAbstractions.get_backend(Y)
    kernel! = axpy_kernel!(backend)
    kernel!(Y, a, X, ndrange = size(Y))
	KernelAbstractions.synchronize(backend)
    return
end

axpy_ka! (generic function with 1 method)

### Diffusion kernel

In [28]:
using OffsetArrays
using CairoMakie

In [29]:
using OrdinaryDiffEqTsit5

In [30]:
a  = 0.01
dx = 0.01  # x-grid spacing
dy = 0.01  # y-grid spacing

0.01

In [31]:
t_final = 3.0

3.0

In [32]:
N = 64

64

Implement a kernel that solves the 2D Diffusion Equation.

$\frac{\partial U}{\partial t} = a * (\frac{\partial^2 U}{\partial x^2} + \frac{\partial^2 U}{\partial y^2})$

Using a finite-difference approximation:

$dU = a * (\frac{U[i+1, j] - 2U[i,j] + U[i-1,j]}{dx^2} + \frac{U[i, j+1] - 2U[i,j] + U[i,j-1]}{dy^2} )$


The code below implements the harness using OffsetArrays and wrap around boundary conditions to make your life easier.

Remember yesterdays serial version.

```julia
for i in 1:N, j in 1:M
	du[i, j] = a * (
		(u[i + 1, j] - 2 * u[i, j] + u[i - 1, j]) / dx^2 +
		(u[i, j + 1] - 2 * u[i, j] + u[i, j - 1]) / dy^2
	)
end
```

In [33]:
@kernel function heat_2D_kernel!(du, @Const(u), a, dx, dy)
	# implement me
end

heat_2D_kernel! (generic function with 4 methods)

In [58]:
function heat_2D!(du, u, (a, dx, dy, xs, ys), t)
	  N, M = size(u)
    N = N - 2
    M = M - 2

	  # update boundary condition (wrap around)
    u[1, :]   .= u[N+1, :]
    u[N+2, :] .= u[2, :]
    u[:, 1]   .= u[:, N+1]
    u[:, N+2] .= u[:, 1]

    # OffsetArray and CUDA don't compose neatly :/
    u = OffsetArray(u, xs, ys)
    du = OffsetArray(u, xs, ys)

    kernel = heat_2D_kernel!(get_backend(du))
    kernel(du, u, a, dx, dy; ndrange=(N,M))

	return nothing
end

heat_2D! (generic function with 1 method)

In [40]:
backend = CUDA.CUDABackend()

CUDA.CUDAKernels.CUDABackend(false, false)

In [49]:
begin
	xs = 0:(N+1)
	ys = 0:(N+1)

	u₀ = KernelAbstractions.zeros(
			backend, Float32, N+2, N+2)
	u₀[16:32, 16:32] .= 5

	nothing
end

In [59]:
prob = ODEProblem(heat_2D!, u₀, (0.0, t_final), (a, dx, dy, xs, ys));

In [56]:
CUDA.allowscalar(false)

In [60]:
sol = solve(prob, Tsit5(), saveat=0.2);

In [44]:
let
	idx = Observable(1)
	data = @lift Array(parent(sol.u[$idx]))
	fig, ax, hm = heatmap(xs, ys, data)

	Makie.Record(fig, 1:length(sol.u), framerate=5) do i
	    idx[] = i
	end
end

VideoStream(Base.PipeEndpoint(RawFD(95) open, 0 bytes waiting), Process(setenv(`/root/.julia/artifacts/0319290fc39dd3bdcb6fb33a4f25d7b8c511c762/bin/ffmpeg -y -loglevel quiet -threads 2 -framerate 5 -pixel_format rgb24 -f rawvideo -s:v 1200x900 -r 5 -i pipe:0 -vf vflip -profile:v high422 -crf 20 -preset slow -c:v libx264 -pix_fmt yuv420p -an '/tmp/jl_KeUwSN/##video#302.mp4'`,["ENV=/root/.bashrc", "PATH=/root/.julia/artifacts/b8a470e4553346122607f94ec60d83c77f061cd6/bin:/root/.julia/artifacts/020f522505f193c968b0a9d6199f7de2c4748cd2/bin:/root/.julia/artifacts/cc3c4d36f2cf650d73959f9d7dec5cd0630bfb01/bin:/root/.julia/artifacts/258f7271ca555a7634c0fe8b8508f69334673a89/bin:/root/.julia/artifacts/c8a20a2030f10b70947d8d2a6bff7f8b5f343fe9/bin:/root/.julia/artifacts/100f53309066fcdee2a61044b118ed3342683b67/bin:/root/.julia/artifacts/bdb4015047b09c6ac5002f73dfd5be8ddf59189b/bin:/root/.julia/artifacts/c9533bea1f28a1adea8cb096009f360534f7ba90/bin:/root/.julia/artifacts/48d69f95e00f33788b0bddee11ce58dd15ba21e5/bin:/root/.julia/artifacts/0319290fc39dd3bdcb6fb33a4f25d7b8c511c762/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin", "NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.22.3-1+cuda12.5", "NV_LIBNCCL_PACKAGE_VERSION=2.22.3-1", "UV_INSTALL_DIR=/usr/local/bin", "COLAB_JUPYTER_IP=172.28.0.12", "NV_LIBCUBLAS_PACKAGE_NAME=libcublas-12-5", "NV_NVPROF_VERSION=12.5.82-1", "NVIDIA_DRIVER_CAPABILITIES=compute,utility", "UV_BUILD_CONSTRAINT="  …  "FONTCONFIG_PATH=/root/.julia/artifacts/258f7271ca555a7634c0fe8b8508f69334673a89/etc/fonts", "NV_LIBCUBLAS_VERSION=12.5.3.2-1", "PYDEVD_USE_FRAME_EVAL=NO", "_PYVIZ_COMMS_INSTALLED=1", "COLAB_LANGUAGE_SERVER_PROXY_LSP_DIRS=/datalab/web/pyright/typeshed-fallback/stdlib,/usr/local/lib/python3.10/dist-packages", "NV_CUDA_NSIGHT_COMPUTE_DEV_PACKAGE=cuda-nsight-compute-12-5=12.5.1-1", "COLAB_FILE_HANDLER_ADDR=localhost:3453", "NV_CUDNN_PACKAGE_DEV=libcudnn9-dev-cuda-12=9.2.1.18-1", "OPENBLAS_MAIN_FREE=1", "MODEL_PROXY_HOST=https://mp.kaggle.net"]), ProcessRunning), CairoMakie.Screen{CairoMakie.IMAGE}(Scene(1 children, 0 plots), Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x000000004b6da510, 1200.0, 900.0, ColorTypes.ARGB32[ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) … ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8); ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) … ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8); … ; ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) … ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8); ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) … ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8)]), Cairo.CairoContext(Ptr{Nothing} @0x000000004debffd0, Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x000000004b6da510, 1200.0, 900.0, ColorTypes.ARGB32[ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) … ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8); ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) … ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8); … ; ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) … ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8); ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) … ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8) ARGB32(1.0N0f8, 1.0N0f8, 1.0N0f8, 1.0N0f8)]), Ptr{Nothing} @0x000000004dcb5710), 2.0, 6, false, CairoMakie.ScreenConfig(2.0, 0.75, :best, false, false, nothing)), Makie.TickControl